In [1]:
import pandas as pd
import numpy as np
import datetime

import GeoFeatureGeneration as gfg

def drop_all_0_row(df):
    # x1 = df[(df==0).all(axis=1)]
    # x2 = 
    return df.drop(index=df[(df==0).all(axis=1)].index)

stay = pd.read_csv('./Data/Output/Stay/153.csv', index_col=0)
stay['stime'] = pd.to_datetime(stay['stime'])
print(stay.columns)

Index(['userID', 'stime', 'grid', 'etime', 'lon', 'lat', 'duration', 'stayid'], dtype='object')


In [2]:
gFeaturePath = './Data/Output/MultipleFeatures/PoIFeature.csv'
PoIFeature = pd.read_csv(gFeaturePath, index_col=0)
PoIFeature['grid'] = PoIFeature.index

stay = stay.merge(PoIFeature, on='grid', how='left').fillna(0)
print(stay.columns)

Index(['userID', 'stime', 'grid', 'etime', 'lon', 'lat', 'duration', 'stayid',
       '0.0', '1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0',
       '10.0', '11.0', '12.0', '13.0'],
      dtype='object')


In [4]:
stay = stay.apply(gfg.GenerateTimeFeature, col='stime', axis=1)
# stay.head()

stay['stimestamp'] = stay['stime'].astype('int64') // 1e9
# stay.head()

s1 = stay.groupby(pd.Grouper(key='stime', freq='M'))

i = 0
maxrow = 128
print(stay.shape)


(3258, 29)


In [8]:
for g in s1:
    df = g[1].copy()
    print(df.columns)

    df.drop(['stime', 'etime', 'lon', 'lat', 'stayid'], axis=1, inplace=True)
    print(df.columns)

Index(['userID', 'stime', 'grid', 'etime', 'lon', 'lat', 'duration', 'stayid',
       '0.0', '1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0',
       '10.0', '11.0', '12.0', '13.0', 'weekofyear', 'dayofweek', 'dayofyear',
       'quarter', 'month', 'hour', 'stimestamp'],
      dtype='object')
Index(['userID', 'grid', 'duration', '0.0', '1.0', '2.0', '3.0', '4.0', '5.0',
       '6.0', '7.0', '8.0', '9.0', '10.0', '11.0', '12.0', '13.0',
       'weekofyear', 'dayofweek', 'dayofyear', 'quarter', 'month', 'hour',
       'stimestamp'],
      dtype='object')
Index(['userID', 'stime', 'grid', 'etime', 'lon', 'lat', 'duration', 'stayid',
       '0.0', '1.0', '2.0', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0',
       '10.0', '11.0', '12.0', '13.0', 'weekofyear', 'dayofweek', 'dayofyear',
       'quarter', 'month', 'hour', 'stimestamp'],
      dtype='object')
Index(['userID', 'grid', 'duration', '0.0', '1.0', '2.0', '3.0', '4.0', '5.0',
       '6.0', '7.0', '8.0', '9.0', '10.0',

In [6]:
result = np.empty((0, maxrow, stay.shape[1]))
# print(result.shape)

# print(s1.shape)
# s1 = drop_all_0_row(s1)
# print(s1.shape)

for g in s1:
    key = g[0].month
    # print(type(key))
    # .to_list()
    # value = g[1][['grid', 'stimestamp', 'duration']].values
    # print(g[1].columns)
    df = g[1].copy()

    # print("before drop all 0 {}".format(df.shape))
    df  = drop_all_0_row(df)
    # print("after drop all 0 {}".format(df.shape))

    value = df.values
    # print(value.shape)
    
    # value.dropna(axis=0, how='all', inplace=True)
    # 去除全零的行。
    # 由于含有datetime格式的数据，所以无法使用下面的语句来去掉全零行。
    # value = value[value.sum(axis=1)!=0,:]
    # 如果行数为0，也就是说没有轨迹点。那么就跳过。
    if value.shape[0] == 0:
        continue
    # 将轨迹填充为相同的形状。
    if value.shape[0] > maxrow:
        continue
    else:
        value = np.pad(array=value, pad_width=((0,maxrow-value.shape[0]),(0,0)), mode='constant')
    
    # print("value {}".format(value.shape))
    # print(result.shape)
    # d = np.concatenate((a[np.newaxis,:], b[np.newaxis,:]), axis=0)
    result = np.concatenate((result, value[np.newaxis,:]), axis=0)
    # if i < 2:
        
    #     print(value)
    #     i += 1
    # print(type(value))
    # output type is <class 'numpy.ndarray'>
    # np.set_printoptions(suppress=True)
    # print(value)
print(result.shape)


(52, 128, 29)


In [9]:
def np_3d_to_csv(data, 
                 path, 
                 datatype='float'):
    import csv
    a2d = data.reshape(data.shape[0], -1)
    with open(path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(a2d)


In [ ]:
np_3d_to_csv(result, "output.csv")

In [8]:
a2d = result.reshape(result.shape[0], -1)
# np.savetxt('{}.csv'.format(153), a2d, delimiter="," )
import csv
with open("output.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(a2d)
print(result.shape)

(52, 128, 29)


In [21]:
import csv
import numpy as np

# 从csv文件读取2D数组
with open("output.csv", "r") as f:
    reader = csv.reader(f)
    a2d = np.array(list(reader)).astype("float")GetAddressByApply

# 将2D数组转换为3D数组
a = a2d.reshape(-1, 128, 3)
print(a.shape)

(52, 128, 3)


In [ ]:
import pandas as pd

def drop_all_0_row(df):
    # x1 = df[(df==0).all(axis=1)]
    # x2 = 
    return df.drop(index=df[(df==0).all(axis=1)].index)

def tte(user):
    stay = pd.read_csv('./Data/Output/Stay/{}.csv'.format(user), index_col=0)
    stay['stime'] = pd.to_datetime(stay['stime'])
    s1 = stay.groupby(pd.Grouper(key='stime', freq='M'))

    gridList = []
    for g in s1:
        key = g[0].month
        # print(type(key))
        value = g[1]['grid'].to_list()
        gridList.append(value)
        # print(key)
        # print(value)
    gridpd = pd.DataFrame(gridList)
    gridpd = drop_all_0_row(gridpd)
    gridpd.dropna(axis=0, how='all', inplace=True)
    print(gridpd.shape)
    return gridpd

cantor 配对函数

In [6]:
import math
def cantor_pairing_function(x, y):
    """
    计算Cantor对数
    :param x: 第一个数
    :param y: 第二个数
    :return: Cantor对数
    """
    return ((x + y) * (x + y + 1) // 2 + y)

def cantor_unpair(z):
    w = (math.sqrt(8 * z + 1) - 1) // 2
    t = w * (w + 1) // 2
    y = z - t
    x = w - y
    return int(x), int(y)

x = 4
y = 7

print(cantor_pairing_function(x, y))

73


In [2]:
a = 7
b = 4
print(cantor_pairing_function(a, b))

70


In [5]:
for x in range(0, 11):
    for y in range(0, 11):
        print(cantor_pairing_function(x, y))

0
2
5
9
14
20
27
35
44
54
65
1
4
8
13
19
26
34
43
53
64
76
3
7
12
18
25
33
42
52
63
75
88
6
11
17
24
32
41
51
62
74
87
101
10
16
23
31
40
50
61
73
86
100
115
15
22
30
39
49
60
72
85
99
114
130
21
29
38
48
59
71
84
98
113
129
146
28
37
47
58
70
83
97
112
128
145
163
36
46
57
69
82
96
111
127
144
162
181
45
56
68
81
95
110
126
143
161
180
200
55
67
80
94
109
125
142
160
179
199
220


In [3]:
x = 4

print(x % 2)

print(x / 2)

0
2.0


In [14]:
x = -7

if x >= 0:
    x = 2 * x
else:
    x = 2 * abs(x) - 1

print(x)

13


In [15]:
x = 13

if x % 2 == 0:
    x = x / 2
else:
    x = - ((x + 1) / 2)
    
print(int(x))

-7


In [20]:
import math
def CantorPairingFunction(x, y):
    """_summary_
    先对x,y使用折叠函数，然后再计算2个数的cantor配对函数的值。
    Args:
        x (int): 整数。
        y (int): 整数。

    Returns:
        int: 返回cantor配对数。
    """
    if x >= 0:
        x = 2 * x
    else:
        x = 2 * abs(x) - 1
    
    if y >= 0:
        y = 2 * y
    else:
        y = 2 * abs(y) - 1

    return ((x + y) * (x + y + 1) // 2 + y)

def CantorPairingInverseFunction(z):
    """_summary_
    先计算cantor配对函数反函数，然后x,y使用折叠反函数。
    Args:
        z (int): 两个数的cantor配对数值。

    Returns:
        x (int): 整数。
        y (int): 整数。
    """
    if z < 0 :
        print('CantorPairingInverseFunction input z is out of range.')
        return 0, 0
    
    w = (math.sqrt(8 * z + 1) - 1) // 2
    t = w * (w + 1) // 2
    y = z - t
    x = w - y
    
    if x % 2 == 0:
        x = x / 2
    else:
        x = -((x + 1) / 2)
    
    if y % 2 == 0:
        y = y / 2
    else:
        y = -((y + 1) / 2)

    return int(x), int(y)

In [27]:
CantorPairingFunction(-2,-1)

11

In [28]:
x,y = CantorPairingInverseFunction(11)
print(x,y)

-2 -1


In [3]:
import pandas as pd
import polars as pl
import numpy as np
import time

df1 = pl.read_csv('./Data/Output/PoiFeature.csv', has_header=True)
df1.head(3)

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,longitude,latitude,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,…,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2541.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,1.0,115.482481,39.65181,0.004289,0.016499,0.003141,-0.00076,-0.009566,-0.008357,-0.091427,0.010202,-0.038074,-0.018119,-0.014617,-0.046119,-0.09011,0.039288,-0.017326,-0.018314,-0.024455,-0.000881,0.012357,-0.02212,…,-0.006907,0.035788,-0.056014,-0.002544,0.047614,-0.005141,-0.001159,0.040743,0.042059,0.006464,0.037266,0.03376,0.01275,-0.014765,0.077446,0.024541,0.045374,-0.072902,-0.102396,-0.043924,0.017011,0.033909,0.07826,0.006621,0.003874,0.05384,-0.016581,0.010054,0.023912,-0.010222,-0.037931,-0.010941,0.003338,-0.01872,0.009969,0.006657,0.043991
2684.0,0.0,0.0,1.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,115.494264,39.65181,0.001293,0.025325,-0.029833,-0.030759,-0.029629,-0.004699,0.008305,-0.001472,-0.04342,-0.024059,-0.044851,0.012433,-0.033121,0.012506,-0.026177,0.009624,-0.02996,-0.011991,-0.016942,0.016075,…,-0.020418,0.028713,0.003886,0.004295,0.024014,0.013459,-0.035204,0.013323,0.035572,0.021832,0.02056,-0.003609,0.015416,0.016945,0.03776,-0.040684,0.046689,-0.06929,-0.037012,-0.025192,0.012876,-0.006196,0.043972,0.02394,-0.004437,0.04758,-0.054088,0.018877,0.05218,-0.004291,-0.028057,0.021559,0.009324,0.019704,0.020393,0.004436,0.020841
2686.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,115.482481,39.660803,0.004289,0.016499,0.003141,-0.00076,-0.009566,-0.008357,-0.091427,0.010202,-0.038074,-0.018119,-0.014617,-0.046119,-0.09011,0.039288,-0.017326,-0.018314,-0.024455,-0.000881,0.012357,-0.02212,…,-0.006907,0.035788,-0.056014,-0.002544,0.047614,-0.005141,-0.001159,0.040743,0.042059,0.006464,0.037266,0.03376,0.01275,-0.014765,0.077446,0.024541,0.045374,-0.072902,-0.102396,-0.043924,0.017011,0.033909,0.07826,0.006621,0.003874,0.05384,-0.016581,0.010054,0.023912,-0.010222,-0.037931,-0.010941,0.003338,-0.01872,0.009969,0.006657,0.043991


In [4]:
df1 = df1.rename({df1.columns[0]: "grid"})
df1.head(3)

grid,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,longitude,latitude,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,…,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2541.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,1.0,115.482481,39.65181,0.004289,0.016499,0.003141,-0.00076,-0.009566,-0.008357,-0.091427,0.010202,-0.038074,-0.018119,-0.014617,-0.046119,-0.09011,0.039288,-0.017326,-0.018314,-0.024455,-0.000881,0.012357,-0.02212,…,-0.006907,0.035788,-0.056014,-0.002544,0.047614,-0.005141,-0.001159,0.040743,0.042059,0.006464,0.037266,0.03376,0.01275,-0.014765,0.077446,0.024541,0.045374,-0.072902,-0.102396,-0.043924,0.017011,0.033909,0.07826,0.006621,0.003874,0.05384,-0.016581,0.010054,0.023912,-0.010222,-0.037931,-0.010941,0.003338,-0.01872,0.009969,0.006657,0.043991
2684.0,0.0,0.0,1.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,115.494264,39.65181,0.001293,0.025325,-0.029833,-0.030759,-0.029629,-0.004699,0.008305,-0.001472,-0.04342,-0.024059,-0.044851,0.012433,-0.033121,0.012506,-0.026177,0.009624,-0.02996,-0.011991,-0.016942,0.016075,…,-0.020418,0.028713,0.003886,0.004295,0.024014,0.013459,-0.035204,0.013323,0.035572,0.021832,0.02056,-0.003609,0.015416,0.016945,0.03776,-0.040684,0.046689,-0.06929,-0.037012,-0.025192,0.012876,-0.006196,0.043972,0.02394,-0.004437,0.04758,-0.054088,0.018877,0.05218,-0.004291,-0.028057,0.021559,0.009324,0.019704,0.020393,0.004436,0.020841
2686.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,115.482481,39.660803,0.004289,0.016499,0.003141,-0.00076,-0.009566,-0.008357,-0.091427,0.010202,-0.038074,-0.018119,-0.014617,-0.046119,-0.09011,0.039288,-0.017326,-0.018314,-0.024455,-0.000881,0.012357,-0.02212,…,-0.006907,0.035788,-0.056014,-0.002544,0.047614,-0.005141,-0.001159,0.040743,0.042059,0.006464,0.037266,0.03376,0.01275,-0.014765,0.077446,0.024541,0.045374,-0.072902,-0.102396,-0.043924,0.017011,0.033909,0.07826,0.006621,0.003874,0.05384,-0.016581,0.010054,0.023912,-0.010222,-0.037931,-0.010941,0.003338,-0.01872,0.009969,0.006657,0.043991


In [6]:
df1 = df1.with_columns(df1["grid"].cast(pl.Int32))
df1.head(3)

grid,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,longitude,latitude,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,…,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2541,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,1.0,115.482481,39.65181,0.004289,0.016499,0.003141,-0.00076,-0.009566,-0.008357,-0.091427,0.010202,-0.038074,-0.018119,-0.014617,-0.046119,-0.09011,0.039288,-0.017326,-0.018314,-0.024455,-0.000881,0.012357,-0.02212,…,-0.006907,0.035788,-0.056014,-0.002544,0.047614,-0.005141,-0.001159,0.040743,0.042059,0.006464,0.037266,0.03376,0.01275,-0.014765,0.077446,0.024541,0.045374,-0.072902,-0.102396,-0.043924,0.017011,0.033909,0.07826,0.006621,0.003874,0.05384,-0.016581,0.010054,0.023912,-0.010222,-0.037931,-0.010941,0.003338,-0.01872,0.009969,0.006657,0.043991
2684,0.0,0.0,1.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,115.494264,39.65181,0.001293,0.025325,-0.029833,-0.030759,-0.029629,-0.004699,0.008305,-0.001472,-0.04342,-0.024059,-0.044851,0.012433,-0.033121,0.012506,-0.026177,0.009624,-0.02996,-0.011991,-0.016942,0.016075,…,-0.020418,0.028713,0.003886,0.004295,0.024014,0.013459,-0.035204,0.013323,0.035572,0.021832,0.02056,-0.003609,0.015416,0.016945,0.03776,-0.040684,0.046689,-0.06929,-0.037012,-0.025192,0.012876,-0.006196,0.043972,0.02394,-0.004437,0.04758,-0.054088,0.018877,0.05218,-0.004291,-0.028057,0.021559,0.009324,0.019704,0.020393,0.004436,0.020841
2686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,115.482481,39.660803,0.004289,0.016499,0.003141,-0.00076,-0.009566,-0.008357,-0.091427,0.010202,-0.038074,-0.018119,-0.014617,-0.046119,-0.09011,0.039288,-0.017326,-0.018314,-0.024455,-0.000881,0.012357,-0.02212,…,-0.006907,0.035788,-0.056014,-0.002544,0.047614,-0.005141,-0.001159,0.040743,0.042059,0.006464,0.037266,0.03376,0.01275,-0.014765,0.077446,0.024541,0.045374,-0.072902,-0.102396,-0.043924,0.017011,0.033909,0.07826,0.006621,0.003874,0.05384,-0.016581,0.010054,0.023912,-0.010222,-0.037931,-0.010941,0.003338,-0.01872,0.009969,0.006657,0.043991
